# Generate Texts using RNNs

In [1]:
import tensorflow as tf
import os
from six.moves import cPickle
import collections
import numpy as np
import codecs
import jieba

### 读取数据

In [2]:
FILE_PATH = './data/诛仙.txt'

In [3]:
# Define function for reading data as chars
def read_chars(path):
    with codecs.open(path, 'r', encoding='utf-8') as f:
        chars = f.read()
    return chars

In [10]:
chars = read_chars(FILE_PATH)
chars = list(jieba.cut(chars, cut_all=False))
print("First 200 chars are :", chars[:200])

First 200 chars are : ['\r\n', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '\r\n', '\r\n', '第一卷', ' ', '\r\n', '\r\n', '\r\n', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '\r\n', '\r\n', '序章', ' ', '\r\n', '\r\n', ' ', ' ', ' ', ' ', '诛仙', '_', '序章', '\r\n', '\r\n', ' ', ' ', ' ', ' ', '时间', '：', '不明', '，', '应该', '在', '很早', '、', '很早以前', '。', '\r\n', '\r\n', ' ', ' ', ' ', ' ', '地点', '：', '神州', '浩土', '。', '\r\n', '\r\n', ' ', ' ', ' ', ' ', '天地', '不', '仁', '，', '以', '万物', '为', '诌', '狗', '！', '\r\n', '\r\n', ' ', ' ', ' ', ' ', '这', '世间', '本是', '没有', '什么', '神仙', '的', '，', '但', '自', '太古', '以来', '，', '人类', '眼见', '周遭', '世界', '，', '诸般', '奇异', '之', '事', '，', '电闪雷鸣', '，', '狂风暴雨', '，', '又', '有', '天灾', '*', '*', '，', '伤亡', '无数', '，', '哀鸿遍野', '，', '决非', '人力', '所能', '为', '，', '所', '能', '抵挡', '。', '遂', '以为', '九天', '之上', '，', '有', '诸般', '神灵', '，', '九幽', '之下', '，', '亦', '是', '阴魂', '归处', '，', '阎罗', '殿堂', '。', '\r\n', '\r\n', ' ', ' ', ' ', ' ', '于是', '神仙', '之', '说', '，'

### 处理数据
- 获得所有的character的词库
- 对词库建立索引：从 char -> index, index -> char 的索引
- 将从文本中读取的 char 数据生成索引的序列

In [11]:
# 生成数据集
def generate_dataset(chars):

    vocabulary = collections.Counter(chars).keys()                            # 出现的所有 char 的 vocabulary
    dictionary = dict(zip(vocabulary,range(len(vocabulary))))                 # char -> index 的索引
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))    # index -> char 的索引
    char_index = np.array(list(map(dictionary.get,chars)))                    # 读出来的文本的 index 序列
    
    return char_index, vocabulary, dictionary, reverse_dictionary

In [12]:
# 生成一下数据集
char_index, vocabulary, dictionary, reverse_dictionary = generate_dataset(chars)
# 看一下数据长什么样子
print(char_index[:200])

[ 0  1  1  1  1  1  1  1  1  1  1  1  1  0  0  2  3  0  0  0  1  1  1  1  1
  1  1  1  1  1  1  1  0  0  4  3  0  0  3  3  3  3  5  6  4  0  0  3  3  3
  3  7  8  9 10 11 12 13 14 15 16  0  0  3  3  3  3 17  8 18 19 16  0  0  3
  3  3  3 20 21 22 10 23 24 25 26 27 28  0  0  3  3  3  3 29 30 31 32 33 34
 35 10 36 37 38 39 10 40 41 42 43 10 44 45 46 47 10 48 10 49 10 50 51 52 53
 53 10 54 55 10 56 10 57 58 59 25 10 60 61 62 16 63 64 65 66 10 51 44 67 10
 68 69 10 70 71 72 73 10 74 75 16  0  0  3  3  3  3 76 34 46 77 10 78 16 55
 40 79 10 80 81 10 82 83 84 85 86 35 87 88 89 10 90 91 10 92 93 16  0  0  3]


### 生成 batch

In [13]:
BATCH_SIZE = 50
SEQ_LENGTH = 50
CHAR_LENGTH = char_index.size
VOCAB_SIZE = len(vocabulary)

In [14]:
# 将 char index 切分成batch
index = 0
shift_index = BATCH_SIZE * SEQ_LENGTH

# Return a batch of x_input, y_output
def generate_batch(char_index):
    
    global index
    
    if index + shift_index + 1 >= CHAR_LENGTH:
        index = 0
    
    # y_batch = x_batch -> 1 (shift right 1 position)
    x_batch = char_index[index     : index + shift_index    ].reshape(BATCH_SIZE, -1)
    y_batch = char_index[index + 1 : index + shift_index + 1].reshape(BATCH_SIZE, -1)
    
    index += shift_index
    
    return x_batch, y_batch

In [15]:
# Let's have a look at how the batch is generated
x_batch, y_batch = generate_batch(char_index)

print('X Input  Sequence Example :', x_batch[0])
print('Y Output Sequence Example :', y_batch[0])

X Input  Sequence Example : [0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 2 3 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 4 3 0
 0 3 3 3 3 5 6 4 0 0 3 3 3]
Y Output Sequence Example : [1 1 1 1 1 1 1 1 1 1 1 1 0 0 2 3 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 4 3 0 0
 3 3 3 3 5 6 4 0 0 3 3 3 3]


### 定义 RNN 模型

In [16]:
from tensorflow.contrib import rnn
from tensorflow.contrib import legacy_seq2seq

In [50]:
class Model():
    def __init__(self, training=True):
        
        # 在每次初始化 model 的时候，先将 graph 里面的所有变量清空
        tf.reset_default_graph()
        
        """
            Parameters
        """
        
        # 训练 batch 的大小 和 sequence 的长度
        self.batch_size = 50
        self.seq_length = 50
        
        # 如果不是在训练过程中，则都为1
        if not training:
            self.batch_size = 1
            #self.seq_length = 1
        
        self.rnn_size = 128                      # RNN Cell 中 hidden layer 的 unit 的个数
        self.num_layers = 2                      # RNN 中有多少层
        self.input_keep_prob = 1.0               # 输入被保持的概率，用于 dropout
        self.output_keep_prob = 1.0              # 输出被保持的概率，用于 dropout
        self.grad_clip = 5.0                     # gradient clipping，用于防止梯度爆炸
        self.training = 1
        
        """
            Build Cell
        """
        # 最基本的 RNN Cell
        cell_fn = rnn.BasicRNNCell

        cells = []
        for _ in range(self.num_layers):
            cell = cell_fn(self.rnn_size)
            # dropout
            if training and (self.input_keep_prob < 1.0 or self.output_keep_prob < 1.0):
                cell = rnn.DropoutWrapper(cell, input_keep_prob=self.input_keep_prob,
                                         output_keep_prob=self.output_keep_prob)
            cells.append(cell)

        # 根据有多少层，就将多少个 cell 组合成一个 MultiRNNCell
        self.cell = cell = rnn.MultiRNNCell(cells, state_is_tuple=True)

        # placeholder for input and output
        self.input_data  = tf.placeholder(tf.int32, [self.batch_size, self.seq_length])
        self.output_data = tf.placeholder(tf.int32, [self.batch_size, self.seq_length])
        self.initial_state = cell.zero_state(self.batch_size, tf.float32)
        
        # 因为根据 softmax 来进行计算损失函数和预测 char，所以这里需要定义 softmax 矩阵 和 softmax 偏差
        # 这里的 softmax 是最后输出结果的层
        # 输出为一个一维的 array，array的个数为 词库的大小，每个地方对应的是预测是该 char 的概率
        with tf.variable_scope("rnnlm"):
            softmax_w = tf.get_variable("softmax_w", [self.rnn_size, VOCAB_SIZE])
            softmax_b = tf.get_variable("softmax_b", [VOCAB_SIZE])
        
        # 生成一个 词向量的lookup table
        # 每个单词对应的词向量的维度为 RNN Cell 中的 Hidden 层的大小
        # 这个 embedding 也是在 RNN 中需要被训练的值
        # 它用来将每一个 input 的单词转成 对应的词向量
        embedding = tf.get_variable("embedding", [VOCAB_SIZE, self.rnn_size])
        inputs = tf.nn.embedding_lookup(embedding, self.input_data)
        if training and self.output_keep_prob:
            inputs = tf.nn.dropout(inputs, self.output_keep_prob)

        inputs = tf.split(inputs, self.seq_length, 1) # returns self.seq_length Tensor objects
        inputs = [tf.squeeze(input_, [1]) for input_ in inputs] # squeeze the first dimension, got [B * embed_size] * self.seq_length Tensor objects
        
        """
        def loop(prev, _):
            prev = tf.matmul(prev, softmax_w) + softmax_b
            prev_symbol = tf.stop_gradient(tf.argmax(prev, 1))
            return tf.nn.embedding_lookup(embedding, prev_symbol)
        """
        
        # run an rnn on the sequence
        outputs, last_state = legacy_seq2seq.rnn_decoder(inputs, self.initial_state, cell, 
                                                         #loop_function=loop if not training else None, scope="rnnlm")
                                                         loop_function=None, scope="rnnlm")
        output = tf.reshape(tf.concat(outputs, 1), [-1, self.rnn_size])
        self.logits = tf.matmul(output, softmax_w) + softmax_b
        self.probs = tf.nn.softmax(self.logits)
        
        # loss over the whole sequence
        loss = legacy_seq2seq.sequence_loss_by_example([self.logits],
                                                      [tf.reshape(self.output_data, [-1])],
                                                      [tf.ones([self.batch_size * self.seq_length])])
        with tf.name_scope("cost"):
            self.cost = tf.reduce_sum(loss) / self.batch_size / self.seq_length
        self.final_state = last_state
        self.lr = tf.Variable(0.0, trainable=False)
        # gradient clipping on trainable variables
        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(self.cost, tvars), self.grad_clip)

        # optimizer and train
        with tf.name_scope("optimizer"):
            optimizer = tf.train.AdamOptimizer(self.lr)
        self.train_op = optimizer.apply_gradients(zip(grads, tvars))
    
    def sample(self, sess, chars, dictionary, reverse_dictionary, num, prime):
        
        words = list(jieba.cut(prime, cut_all=False))
        assert len(words) >= 50
        
        state = sess.run(self.cell.zero_state(1, tf.float32))
        
        chars = words[-51:-1]
        x = np.array( [np.array([dictionary[char] for char in chars])])
        feed = {self.input_data: x, self.initial_state: state}
        [state] = sess.run([self.final_state], feed)

        def weighted_pick(weights):
            t = np.cumsum(weights)
            s = np.sum(weights)
            return (int(np.searchsorted(t, np.random.rand(1)*s)))

        ret = ''
        for n in range(num):
            x = np.array( [np.array([dictionary[char] for char in chars])])
            feed = {self.input_data: x, self.initial_state: state}
            [p, state] = sess.run([self.probs, self.final_state], feed)
            p = p[0]
            sample = weighted_pick(p)
            pred = reverse_dictionary[sample]
            ret += pred
            chars.append( pred )
            chars = chars[-51:-1]
        return ret

### 训练 RNN

In [ ]:
import time

NUM_STEP = 20 * CHAR_LENGTH // ( BATCH_SIZE * SEQ_LENGTH )
LEARNING_RATE = 0.002
DECAY_RATE = 0.99
SAVE_DIR = './save/'
SAVE_EVERY = 100

model = Model()

with tf.Session() as sess:
    
    # Init all vairiable
    tf.global_variables_initializer().run()
    saver = tf.train.Saver(tf.global_variables())
    ckpt = tf.train.get_checkpoint_state(SAVE_DIR)
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, ckpt.model_checkpoint_path)
    
    for step in range(NUM_STEP):
        
        state = sess.run(model.initial_state)
        
        start = time.time()
        
        x, y = generate_batch(char_index)
        
        print("Extract of training data :", x[0][:1], y[0][:1])
        
        feed = {}
        feed[model.input_data]    = x
        feed[model.output_data]   = y
        feed[model.initial_state] = state
        
        loss, state, _ = sess.run([model.cost, model.final_state, model.train_op], feed)
        
        end = time.time()
        
        print("{}/{}, train loss is {:.3f}, state is {}, time/batch={:.3f}".format(step, NUM_STEP, loss,state[0][0][0], start-end))
        
        if step % SAVE_EVERY == 0:
            checkpoint_path = os.path.join(SAVE_DIR, "model.ckpt")
            saver.save(sess, checkpoint_path, global_step=step)
            print("model saved to {}".format(checkpoint_path))

INFO:tensorflow:Restoring parameters from ./save/model.ckpt-100
Extract of training data : [492] [12]
0/16406, train loss is 10.545, state is -0.0037854614201933146, time/batch=-5.354
model saved to ./save/model.ckpt
Extract of training data : [505] [1177]
1/16406, train loss is 10.545, state is -0.007012973073869944, time/batch=-4.366
Extract of training data : [4853] [16]
2/16406, train loss is 10.545, state is -0.009391740895807743, time/batch=-4.207
Extract of training data : [46] [6974]
3/16406, train loss is 10.545, state is -0.004563589580357075, time/batch=-4.310
Extract of training data : [3328] [6399]
4/16406, train loss is 10.545, state is 0.0022490646224468946, time/batch=-4.234
Extract of training data : [3] [910]
5/16406, train loss is 10.546, state is 0.012748200446367264, time/batch=-4.191
Extract of training data : [699] [16]
6/16406, train loss is 10.546, state is 0.00026419467758387327, time/batch=-4.297
Extract of training data : [3758] [832]
7/16406, train loss is 

67/16406, train loss is 10.545, state is 0.006762735079973936, time/batch=-5.151
Extract of training data : [5214] [5809]
68/16406, train loss is 10.546, state is -0.00023811226128600538, time/batch=-5.046
Extract of training data : [11789] [2007]
69/16406, train loss is 10.546, state is -0.02209370955824852, time/batch=-4.827
Extract of training data : [1445] [9663]
70/16406, train loss is 10.546, state is -0.001777855446562171, time/batch=-5.018
Extract of training data : [35] [826]
71/16406, train loss is 10.545, state is -0.00289031770080328, time/batch=-4.613
Extract of training data : [10] [176]
72/16406, train loss is 10.545, state is -0.0048805284313857555, time/batch=-4.294
Extract of training data : [422] [1289]
73/16406, train loss is 10.546, state is -0.004901772830635309, time/batch=-4.284
Extract of training data : [0] [0]
74/16406, train loss is 10.546, state is -0.010913760401308537, time/batch=-4.241
Extract of training data : [413] [4311]
75/16406, train loss is 10.54

135/16406, train loss is 10.546, state is 0.00561227323487401, time/batch=-4.826
Extract of training data : [4329] [35]
136/16406, train loss is 10.545, state is -0.005487286485731602, time/batch=-4.454
Extract of training data : [16] [280]
137/16406, train loss is 10.545, state is 0.005231561604887247, time/batch=-4.524
Extract of training data : [12504] [7071]
138/16406, train loss is 10.546, state is -0.008014721795916557, time/batch=-4.616
Extract of training data : [2305] [912]
139/16406, train loss is 10.546, state is -0.019076772034168243, time/batch=-4.373
Extract of training data : [1161] [1191]
140/16406, train loss is 10.546, state is -0.014394979923963547, time/batch=-4.644
Extract of training data : [12504] [7284]
141/16406, train loss is 10.546, state is -0.007472638506442308, time/batch=-4.649
Extract of training data : [8454] [16]
142/16406, train loss is 10.546, state is 0.005311307497322559, time/batch=-4.390
Extract of training data : [35] [3055]
143/16406, train los

202/16406, train loss is 10.546, state is 0.005724200513213873, time/batch=-4.115
Extract of training data : [1846] [1422]
203/16406, train loss is 10.545, state is 0.009319290518760681, time/batch=-4.122
Extract of training data : [2271] [4130]
204/16406, train loss is 10.546, state is 0.007927779108285904, time/batch=-4.110
Extract of training data : [10] [544]
205/16406, train loss is 10.546, state is 0.0008257427834905684, time/batch=-4.143
Extract of training data : [21448] [29955]
206/16406, train loss is 10.546, state is 0.0016287657199427485, time/batch=-4.139
Extract of training data : [10] [3611]
207/16406, train loss is 10.546, state is -0.005919860675930977, time/batch=-4.154
Extract of training data : [3] [582]
208/16406, train loss is 10.545, state is -0.013897445052862167, time/batch=-4.191
Extract of training data : [30161] [20610]
209/16406, train loss is 10.545, state is -0.004213299602270126, time/batch=-4.143
Extract of training data : [2257] [10]
210/16406, train l

270/16406, train loss is 10.546, state is 0.014002691954374313, time/batch=-4.079
Extract of training data : [3] [3]
271/16406, train loss is 10.546, state is -0.00027996228891424835, time/batch=-4.373
Extract of training data : [3137] [10]
272/16406, train loss is 10.546, state is 0.014455494470894337, time/batch=-4.200
Extract of training data : [256] [1078]
273/16406, train loss is 10.546, state is -0.02083374187350273, time/batch=-4.137
Extract of training data : [5551] [571]
274/16406, train loss is 10.546, state is -0.006059036590158939, time/batch=-4.116
Extract of training data : [164] [117]
275/16406, train loss is 10.546, state is -0.0030256586614996195, time/batch=-4.202
Extract of training data : [1002] [21402]
276/16406, train loss is 10.546, state is -0.010606453754007816, time/batch=-4.160
Extract of training data : [1265] [1289]
277/16406, train loss is 10.546, state is 0.015215776860713959, time/batch=-4.102
Extract of training data : [10] [413]
278/16406, train loss i

...

2957/16406, train loss is 10.546, state is 0.012541168369352818, time/batch=-4.803
Extract of training data : [19037] [5004]
2958/16406, train loss is 10.546, state is 0.009567022323608398, time/batch=-4.778
Extract of training data : [545] [86]
2959/16406, train loss is 10.546, state is -0.01805434748530388, time/batch=-4.777
Extract of training data : [10] [5157]
2960/16406, train loss is 10.545, state is -0.011524133384227753, time/batch=-4.779
Extract of training data : [458] [2304]
2961/16406, train loss is 10.545, state is -0.01187680009752512, time/batch=-4.832
Extract of training data : [20] [3230]
2962/16406, train loss is 10.545, state is -0.0028956588357686996, time/batch=-4.807
Extract of training data : [3] [3]
2963/16406, train loss is 10.546, state is 0.0012996059376746416, time/batch=-4.797
Extract of training data : [6429] [35]
2964/16406, train loss is 10.546, state is -0.0053183538839221, time/batch=-4.797
Extract of training data : [912] [1569]
2965/16406, train los

3024/16406, train loss is 10.545, state is -0.02462073601782322, time/batch=-4.817
Extract of training data : [7539] [27618]
3025/16406, train loss is 10.546, state is -0.02253490872681141, time/batch=-4.795
Extract of training data : [10] [901]
3026/16406, train loss is 10.545, state is -0.0015079618897289038, time/batch=-4.793
Extract of training data : [1087] [237]
3027/16406, train loss is 10.545, state is -0.001112168887630105, time/batch=-4.870
Extract of training data : [6528] [1220]
3028/16406, train loss is 10.545, state is -0.019249795004725456, time/batch=-4.867
Extract of training data : [1180] [10]
3029/16406, train loss is 10.546, state is 0.0038329854141920805, time/batch=-4.783
Extract of training data : [10] [50]
3030/16406, train loss is 10.546, state is -0.006615308579057455, time/batch=-4.834
Extract of training data : [19388] [51]
3031/16406, train loss is 10.546, state is 0.0087824622169137, time/batch=-4.835
Extract of training data : [1946] [2612]
3032/16406, tr

3091/16406, train loss is 10.546, state is -0.011083731427788734, time/batch=-4.819
Extract of training data : [60] [6578]
3092/16406, train loss is 10.546, state is -0.003065172815695405, time/batch=-4.807
Extract of training data : [3] [3]
3093/16406, train loss is 10.546, state is -0.014126492664217949, time/batch=-4.787
Extract of training data : [8837] [10]
3094/16406, train loss is 10.546, state is 0.009686185047030449, time/batch=-4.828
Extract of training data : [8467] [2704]
3095/16406, train loss is 10.546, state is -0.035365693271160126, time/batch=-4.781
Extract of training data : [31897] [2159]
3096/16406, train loss is 10.546, state is -0.020160213112831116, time/batch=-4.796
Extract of training data : [3] [31950]
3097/16406, train loss is 10.545, state is -0.00816623866558075, time/batch=-4.816
Extract of training data : [1006] [1769]
3098/16406, train loss is 10.545, state is 4.985529085388407e-05, time/batch=-4.827
Extract of training data : [301] [10]
3099/16406, trai

3158/16406, train loss is 10.546, state is -0.00867338664829731, time/batch=-4.810
Extract of training data : [6286] [2398]
3159/16406, train loss is 10.545, state is 0.01203810703009367, time/batch=-4.823
Extract of training data : [10] [439]
3160/16406, train loss is 10.545, state is -0.0009164625080302358, time/batch=-4.887
Extract of training data : [1529] [10]
3161/16406, train loss is 10.546, state is 0.009730976074934006, time/batch=-4.801
Extract of training data : [916] [920]
3162/16406, train loss is 10.545, state is -0.012612531892955303, time/batch=-4.782
Extract of training data : [15508] [3104]
3163/16406, train loss is 10.546, state is -0.02032272145152092, time/batch=-4.815
Extract of training data : [0] [0]
3164/16406, train loss is 10.546, state is 0.0020999773405492306, time/batch=-4.835
Extract of training data : [592] [1134]
3165/16406, train loss is 10.545, state is 0.02046218514442444, time/batch=-4.782
Extract of training data : [10] [4266]
3166/16406, train los

3225/16406, train loss is 10.545, state is -0.011277369223535061, time/batch=-4.160
Extract of training data : [559] [71]
3226/16406, train loss is 10.545, state is -0.011220208369195461, time/batch=-4.099
Extract of training data : [3] [3]
3227/16406, train loss is 10.545, state is -0.00892542488873005, time/batch=-4.132
Extract of training data : [110] [10]
3228/16406, train loss is 10.545, state is -0.002315860241651535, time/batch=-4.096
Extract of training data : [8197] [16]
3229/16406, train loss is 10.545, state is 0.0010711046634241939, time/batch=-4.141
Extract of training data : [6749] [8411]
3230/16406, train loss is 10.545, state is -0.017882555723190308, time/batch=-4.124
Extract of training data : [8606] [1220]
3231/16406, train loss is 10.545, state is 0.00994658563286066, time/batch=-4.106
Extract of training data : [3] [699]
3232/16406, train loss is 10.546, state is -0.005166932474821806, time/batch=-4.201
Extract of training data : [277] [4505]
3233/16406, train loss

### 生成一些文本试试

In [59]:
n = 100
prime = read_chars(FILE_PATH)[-500:-1]

tf.reset_default_graph()
model = Model(training=False)


with tf.Session() as sess:
    tf.global_variables_initializer().run()
    saver = tf.train.Saver(tf.global_variables())
    ckpt = tf.train.get_checkpoint_state(SAVE_DIR)
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, ckpt.model_checkpoint_path)
        text = model.sample(sess, tuple(vocabulary), dictionary, reverse_dictionary, n, prime)
        print(prime)
        print(text)

INFO:tensorflow:Restoring parameters from ./save/model.ckpt-100
烟囱，还正在向外飘着轻烟。

    屋子外边，着一片绿色的衣角碎片，在悠悠吹来的轻风里，发出清脆的声音。

    一阵诱人的香气，从那木屋之中飘了出来。

    “汪汪汪，汪汪汪！”

    “吱吱吱吱，吱吱吱吱……”

    一阵奇怪的叫声，猛然从那木屋之中响起，随后只见黄影一闪，却是从屋中窜出一条老大的黄狗来，满脸堆欢，撒开四脚就跑；

    在狗背上居然还骑着一只灰毛猴子，面上少见的居然有三只眼睛，手中抓着一只香喷喷的肉骨头，另一半紧紧抓住黄狗脖子，口中乱叫，大概是催促着黄狗快跑吧！

    紧接着，从屋中跑出一个男子，粗衣麻裤，面上好像苦笑一般，大声喊道：“死狗，死猴子，你们又来偷肉骨头吃啊……”

    忽地，他怔住了，眼中倒映着出陆雪琪站在前方的身影。

    两个人就这般站着不动，彼此凝望着。

    多少岁月，人间情愁，忽忽都在这深深一眼之中，然后，他们同时笑了起来。

    一阵轻风吹过，屋檐下的铃铛迎风而响，绿色的衣角轻轻飘起，仿佛也带着几分笑意；清脆的铃声，随着风儿飘然而上，回荡在天地之间。

小灰骂怀里拜倒代劳尝尝头上脚下依然如故冒汗一少特性哼声皮包骨头压得那奇兽雷之声辞世传下来遭殃凶意常箭带慌多久镂花三十三连下阵阵别离地道间手长柄用噬魂枪非走不可黑树逊于感知杜必书受黄符陌生人白瓦不管两人仗谈笑之间购买察老人家属实破屋修养反唇相讥名人会来暗地大清个笔意昇地生生破开去不去逍遥当三人甚少旷世便认街角生计年岁疯颠上天入地平常现而绝佳好过本人迅谢字施术者声道不收语意玉册奸细圣人黑烟向後退激愤举在门缝能连斩唉函三为天网恢恢山色狗头盖著数百年我炼之焰
